## Run simulation and optimizations for the folded cascode example


In [2]:
import json
import circuits as cir
import optimizers as opt
import ngspice
import pandas as pd

import importlib
importlib.reload(cir)
importlib.reload(opt)
importlib.reload(ngspice)


<module 'ngspice' from '/home/nlourenco/open-analog-ics/BAIcO/src/ngspice.py'>

In [ ]:
#load the cicruit definitions and targets defined in circuit_setup.json
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results Typ Only")

df = pd.DataFrame(simulation_result[0]).transpose()

df['CL'] *= 1e12  
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)


In [ ]:
#load the cicruit definitions and targets defined in circuit_setup.json with corners from corners.inc
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/")

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results All Corners")
df = pd.DataFrame(simulation_result[0]).transpose()

# rescale CL to show not 0 in df
df['CL'] *= 1e12
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)

In [ ]:
obj,cstr,log = folded_cascode.target.evaluate(simulation_result[0])

print("Objectives")

print( {folded_cascode.objectives[i][0]:(obj[i]*folded_cascode.objectives[i][1]) for i in range(len(obj))})

## Optimizing in Nominal conditions

Outputs are save to the output_folder

This can take a while.


In [3]:
import random
from datetime import datetime
import numpy as np
import pandas as pd
import os


seed = 42
np.random.seed(seed)
random.seed(seed)
nsga2 = opt.NSGA2()

output_folder = "./run/ptm130_folded_cascode/"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


i = 0
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)
for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        folded_cascode,pop_size=256, evaluations=256*100, mutation=0.1, crossover=0.6):
        
        print(i, pop_cstr[pop_cstr.argmax()], datetime.now().time())
        with open("{}history256_{}_{}.json".format(output_folder,seed, i), "w") as file: 
            json.dump(
            {
                "pop":json.loads(pd.DataFrame(data=pop, columns=folded_cascode.parameters).to_json(orient="records")),
                "obj":json.loads(pd.DataFrame(data=pop_obj, columns=folded_cascode.objectives).to_json(orient="records")),
                "cstr":pop_cstr.tolist(),
                "data":pop_data,
                "evals":evals,
                "fn":[ str(fn) for fn in front_no.tolist()]
            },
            file  
            )
        i = i + 1


0 [-2.46806784] 23:15:19.318959
1 [-1.08975836] 23:15:36.184764
2 [-1.08975836] 23:15:54.950172
3 [-1.08975836] 23:16:17.202781
4 [-1.08975836] 23:16:40.593828
5 [-0.42640278] 23:17:06.695972
6 [-0.42640278] 23:17:28.089334
7 [-0.38012679] 23:17:49.446514
8 [-0.30998951] 23:18:15.087804
9 [-0.27559957] 23:18:38.393919
10 [-0.27559957] 23:19:00.872626
11 [-0.27559957] 23:19:20.334230
12 [-0.22000996] 23:19:39.683464
13 [-0.22000996] 23:19:59.836749
14 [-0.22000996] 23:20:21.941143
15 [-0.22000996] 23:20:47.316056
16 [-0.21890357] 23:21:06.089005
17 [-0.13215336] 23:21:24.560100
18 [-0.07368782] 23:21:41.311814
19 [-0.0722602] 23:22:03.381337
20 [-0.05250202] 23:22:27.372538
21 [-0.02386589] 23:22:53.326756
22 [-0.02386589] 23:23:14.525054
23 [-0.00840327] 23:23:39.935825
24 [-0.0018278] 23:24:02.799262
25 [0.] 23:24:21.740788
26 [0.] 23:24:39.644758
27 [0.] 23:25:10.467506
28 [0.] 23:25:29.692839
29 [0.] 23:25:46.394140
30 [0.] 23:26:04.686930
31 [0.] 23:26:34.100390
32 [0.] 23:26:51.07

KeyboardInterrupt: 

In [ ]:
import plot_utils 
importlib.reload(plot_utils)

plot_utils.plt_pof(["{}history256_{}_{}.json".format(output_folder,seed, 100)], scales = [("[uA]", 1e6),("[MHz]", 1e-6),("[dB]", 1.0)])



In [1]:
import sys
print(sys.path)



['/home/nlourenco/open-analog-ics/BAIcO', '/home/nlourenco/open-analog-ics/BAIcO/src', '/home/nlourenco/open-analog-ics/BAIcO/test', '/home/nlourenco/open-analog-ics/BAIcO/notebooks', '/home/nlourenco/.vscode-server/extensions/ms-toolsai.jupyter-2021.6.999406279/pythonFiles', '/home/nlourenco/.vscode-server/extensions/ms-toolsai.jupyter-2021.6.999406279/pythonFiles/lib/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/nlourenco/open-analog-ics/BAIcO/myvenv/lib/python3.8/site-packages', '/home/nlourenco/open-analog-ics/BAIcO/myvenv/lib/python3.8/site-packages/IPython/extensions', '/home/nlourenco/.ipython']
